In [1]:
import numpy as np
import pandas as pd
#import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from keras.utils import np_utils

import matplotlib.pyplot as plt
import random

Using TensorFlow backend.


In [2]:
r = pd.read_csv('dsf2.csv')

In [3]:
from sklearn.model_selection import train_test_split
x = r['Content']
y = r['Label']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=123)

In [4]:
x_train = x_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [5]:
f = open('glove.6B.50d.txt')

In [6]:
for ix in range(1, x_train.shape[0]):
    x_train[ix] = x_train[ix].split()
    
for ix in range(1, x_test.shape[0]):
    x_test[ix] = x_test[ix].split()



In [7]:
#dictionary 
embeddings_index = {}
cnt = 0
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype = 'float')
    embeddings_index[word] = coefs
f.close()

In [8]:
embeddings_index['eat']

array([ 6.4295e-01, -4.2946e-01, -5.4277e-01, -1.0307e+00,  1.2056e+00,
       -2.7174e-01, -6.3561e-01, -1.5065e-02,  3.7856e-01,  4.6474e-02,
       -1.3102e-01,  6.0500e-01,  1.6391e+00,  2.3940e-01,  1.2128e+00,
        8.3178e-01,  7.3893e-01,  1.5200e-01, -1.4175e-01, -8.8384e-01,
        2.0829e-02, -3.2545e-01,  1.8035e+00,  1.0045e+00,  5.8484e-01,
       -6.2031e-01, -4.3296e-01,  2.3562e-01,  1.3027e+00, -8.1264e-01,
        2.3158e+00,  1.1030e+00, -6.0608e-01,  1.0101e+00, -2.2426e-01,
        1.8908e-02, -1.0931e-01,  3.8350e-01,  7.7362e-01, -8.1927e-02,
       -3.4040e-01, -1.5143e-03, -5.6640e-02,  8.7359e-01,  1.4805e+00,
        6.9421e-01, -3.0966e-01, -9.0826e-01,  3.7277e-03,  8.4550e-01])

In [9]:
emb_dim = embeddings_index["eat"].shape[0]
def embedding_output(X):
    embedding_matrix = np.zeros((X.shape[0], maxLen, emb_dim))
    for word, index in tokenizer.word_index.items():
        if index > vocabulary_size - 1:
            break
        else:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[index] = embedding_vector
    return embedding_out

In [10]:
def embedding_output(X):
    maxLen = 5000
    embedding_out = np.zeros((X.shape[0], maxLen, emb_dim))
    
    #for ix in range(X.shape[0]):
       # X[ix] = X[ix].split()
    
    for ix in range(1, X.shape[0]):
        for ij in range(1, len(X[ix])):
            try:
                embedding_out[ix][ij] = embeddings_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij] = np.zeros((50, ))
    return embedding_out

In [11]:
embeddings_matrix_train = embedding_output(x_train)
embeddings_matrix_test = embedding_output(x_test)

In [12]:
from keras.utils.np_utils import to_categorical

In [13]:
y_train = to_categorical((y_train), 4)
y_test = to_categorical((y_test),4)


In [14]:
print (embeddings_matrix_train.shape, embeddings_matrix_test.shape)

(1428, 5000, 50) (358, 5000, 50)


In [15]:
from keras.models import Sequential
from keras.layers import *

In [16]:
model = Sequential()
model.add(LSTM(64, input_shape=(5000,50)))
#model.add(Dropout(0.5))
#model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))
model.compile(loss ='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0         
Total params: 29,700
Trainable params: 29,700
Non-trainable params: 0
_________________________________________________________________


In [17]:
from keras import optimizers
model.compile(loss='categorical_crossentropy', optimizer=optimizers.adam(learning_rate = 0.1), metrics=['accuracy'])

In [23]:
hist = model.fit(embeddings_matrix_train,y_train, epochs = 100, batch_size = 64,shuffle=True, validation_split = 0.2)

Train on 1142 samples, validate on 286 samples
Epoch 1/100
1142/1142 [==============================] - 114s 100ms/step - loss: 0.9161 - accuracy: 0.6848 - val_loss: 0.6396 - val_accuracy: 0.7587
Epoch 2/100
1142/1142 [==============================] - 109s 95ms/step - loss: 0.8890 - accuracy: 0.6550 - val_loss: 0.6455 - val_accuracy: 0.7483
Epoch 3/100
1142/1142 [==============================] - 106s 93ms/step - loss: 0.8468 - accuracy: 0.6795 - val_loss: 0.6510 - val_accuracy: 0.7622
Epoch 4/100
1142/1142 [==============================] - 121s 106ms/step - loss: 0.8054 - accuracy: 0.6874 - val_loss: 0.7417 - val_accuracy: 0.7413
Epoch 5/100
1142/1142 [==============================] - 115s 101ms/step - loss: 0.8180 - accuracy: 0.6883 - val_loss: 0.7163 - val_accuracy: 0.7483
Epoch 6/100
1142/1142 [==============================] - 118s 104ms/step - loss: 0.7613 - accuracy: 0.7163 - val_loss: 0.6612 - val_accuracy: 0.7587
Epoch 7/100
1142/1142 [==============================] - 118s

Epoch 56/100
1142/1142 [==============================] - 108s 94ms/step - loss: 0.3415 - accuracy: 0.8818 - val_loss: 0.1986 - val_accuracy: 0.9510
Epoch 57/100
1142/1142 [==============================] - 109s 95ms/step - loss: 0.3133 - accuracy: 0.9124 - val_loss: 0.1933 - val_accuracy: 0.9441
Epoch 58/100
1142/1142 [==============================] - 108s 95ms/step - loss: 0.2749 - accuracy: 0.9177 - val_loss: 0.2415 - val_accuracy: 0.9441
Epoch 59/100
1142/1142 [==============================] - 108s 95ms/step - loss: 0.2399 - accuracy: 0.9282 - val_loss: 0.2269 - val_accuracy: 0.9510
Epoch 60/100
1142/1142 [==============================] - 108s 95ms/step - loss: 0.2344 - accuracy: 0.9247 - val_loss: 0.1729 - val_accuracy: 0.9615
Epoch 61/100
1142/1142 [==============================] - 114s 100ms/step - loss: 0.2515 - accuracy: 0.9212 - val_loss: 0.2591 - val_accuracy: 0.9371
Epoch 62/100


KeyboardInterrupt: 

In [28]:
pred = model.predict_classes(embeddings_matrix_test)

In [29]:
print(pred)

[2 0 1 0 3 3 0 1 1 0 1 0 0 0 0 1 0 0 3 0 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 1 3
 0 0 3 0 1 1 0 1 1 1 1 0 0 1 0 3 0 1 0 3 0 0 0 0 0 1 0 1 1 3 0 1 1 1 0 0 0
 1 3 0 3 2 0 3 3 1 1 1 0 1 0 1 1 3 0 3 0 0 3 3 3 1 1 0 0 0 0 0 0 0 1 1 1 2
 3 1 1 1 0 0 2 1 1 0 1 1 1 0 0 0 3 0 0 0 0 3 1 0 1 0 0 1 1 0 0 0 3 3 1 3 0
 0 3 0 0 1 0 1 3 0 0 0 1 2 0 1 0 0 1 0 3 0 3 0 0 1 0 3 1 0 0 0 0 0 2 1 0 0
 3 1 0 1 0 0 3 3 1 0 0 0 1 2 0 0 3 0 1 0 3 1 3 0 1 0 0 0 0 3 1 1 1 1 1 0 1
 0 1 0 1 1 0 1 1 0 0 1 1 0 1 0 0 1 1 1 3 3 0 2 1 1 0 2 1 0 1 1 0 1 3 1 0 0
 0 1 1 3 1 1 3 0 1 0 3 0 1 1 0 0 0 3 0 0 1 1 1 3 0 0 1 1 1 0 3 0 0 0 3 0 0
 1 1 0 0 0 0 3 3 1 1 1 1 0 0 0 0 3 0 0 0 0 1 1 3 0 0 1 1 1 1 1 1 3 0 0 0 1
 1 0 0 1 0 3 1 1 1 1 3 1 2 0 0 1 0 0 1 1 0 0 1 0 0]


In [30]:
model.evaluate(embeddings_matrix_test,y_test)

358/358 [==============================] - 11s 31ms/step


[0.25720409158221835, 0.9357541799545288]

In [31]:
model.evaluate(embeddings_matrix_train,y_train)

1428/1428 [==============================] - 42s 29ms/step


[0.13410216998648034, 0.9586834907531738]